In [6]:
import xml.etree.ElementTree as ET
from datetime import datetime
from googlesearch import search
import time
import requests
from bs4 import BeautifulSoup
import json
import re

In [63]:
def remove_prefix_from_name(name):
    pattern = r"[a-z]{2,}\.\s"
    prefixes = re.findall(pattern, name, flags=re.IGNORECASE)
    
    if prefixes:
        name_without_prefixes = re.sub(pattern, '', name, flags=re.IGNORECASE).strip()
        
        if 'IJ. ' in prefixes:
            prefixes = [i for i in prefixes if 'IJ.' not in i]
            name_without_prefixes = 'IJ. ' + name_without_prefixes
        
        return prefixes, name_without_prefixes
    else:
        return '', name

In [64]:
def xpath(element, path, namespaces):
    obj = element.find(path, namespaces)
    
    if obj is not None:
        return obj.text
    else:
        return ''

In [86]:
def xpath_TOOI(element, path, namespaces, afkorting):
    
    dc_publisher = element.find(path, namespaces)
    if dc_publisher is not None:
        
        # hier wordt de afkorting gebruikt bijv. ws = waterschappen
        if dc_publisher.text.rsplit('/')[-1][:2] == afkorting:
            
            return dc_publisher.text.rsplit('/')[-1]
        else:
            return ''
    else:
        return ''

In [66]:
def xpath_naam(element, path, namespaces):
    # verzamel de naam in de XML
    initial_name = element.find(path, namespaces)
    
    if initial_name is not None:
        initial_name_txt = initial_name.text

        # er staat vaak een prefix als dr. of mew. voor deze wordt verwijderd?
        prefixes, name_without_prefix = remove_prefix_from_name(initial_name_txt)
        name_without_prefix = name_without_prefix.lstrip()

        initial_name_split = name_without_prefix.split(' ', 1)

        # bij gemeenten staat er een persoon in als mew. Schouten dus zonder initialen. 
        if len(initial_name_split) == 2:
            first_word = initial_name_split[0]
            if '.' not in first_word and len(first_word) != 1:

                foaf_firstName = initial_name_split[0]
                foaf_initials = foaf_firstName[0].upper() + '.'
            else:
                foaf_initials = initial_name_split[0]
                foaf_firstName = ''

            foaf_lastName = initial_name_split[1]
        
        else:
            foaf_initials = ''
            foaf_firstName = ''
            foaf_lastName = initial_name_split[0]
    
    # gebeurt in de praktijk niet maar maakt het wel failproof
    else:
        foaf_initials = ''
        foaf_firstName = ''
        foaf_lastName = ''
        
    prefixes = ''.join(prefixes)
    prefixes = prefixes.rstrip()   
            
    return foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes

TODO: van tree automatisch naar namespaces gaan.

In [125]:
def json_create(organisatie_type):
    
    list_of_dicts = []
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Waterschappen': 'ws',
                  'Gemeenten': 'gm',}
    afkorting = afkortingen[organisatie_type]
    
    tree = ET.parse(f"exportOO_{organisatie_type.lower()}.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]

    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    organisatie_elements = tree.findall('.//p:organisatie', namespaces)

    # loop over deze elementen en verzamel gegevens
    for organisatie in organisatie_elements:

        # In de resourceIdentifierTOOI staat de organisatiecode en veramel deze code
        dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 
        
        dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

        website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url''p:types/p:type', namespaces)

        Type = xpath(organisatie, 'p:types/p:type', namespaces)

        # in het functie_element staan alle verschillende functies
        functie_element = organisatie.findall('p:functies/p:functie', namespaces)

        # loop over deze functies
        for functie in functie_element:
            foi_count = 0
            DICT = dict()

            foaf_function_type = xpath(functie, 'p:naam', namespaces)

            # vindt alle medewerkers die bij deze functie horen
            medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

            # loop over alle medewerkers om per medewerker gegevens te verzamelen
            for medewerker in medewerker_element:
                
                foi_party = xpath(medewerker, 'p:partijLidmaatschap', namespaces)

                foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes = xpath_naam(medewerker, 'p:naam', namespaces)

                # optioneel startdatum, telefoonnummer en mailadres
                foi_startDate = xpath(medewerker, 'p:startDatum', namespaces)

                foaf_phone = xpath(medewerker, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

                foaf_mbox = xpath(medewerker, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)

                if organisatie_type == 'Waterschappen':
                    bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {Type.lower()} {dc_publisher_name}"
                elif organisatie_type == 'Gemeenten':
                    if foi_party != '':
                        bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {foi_party} in de {dc_publisher_name}"
                    else:
                        bereikbaarheidsgegevens = f"bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {dc_publisher_name}"
                
                # vul de dictionary in
                Dict = {
                        'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                        'dc_title': f"{name_without_prefix} - {dc_publisher_name}",
                        'foi_function': foaf_function_type,
                        'dc_description': bereikbaarheidsgegevens,
                        'dc_source': f"https://organisaties.overheid.nl/archive/exportOO{organisatie_type.lower()}_.xml",
                        'dc_publisher': dc_publisher,
                        'dc_creator': "Ramon Duursma",
                        'foi_retrievedDate': foi_retrievedDate,
                        'dc_date_year': dc_date_year,
                        'dc_publisher_name': dc_publisher_name,
                        'foaf_initials': foaf_initials,
                        'foaf_firstName': foaf_firstName,
                        'foaf_lastName': foaf_lastName,
                        'foi_title': prefixes,
                        'foaf_name': name_without_prefix,
                        'foaf_mbox': foaf_mbox,
                        'foaf_phone': foaf_phone,
                        'foaf_type': Type,
                        'foi_startDate': foi_startDate,
                        'foi_party': foi_party,
                        'foi_files': [] ,
                    }

                # Verwijder alle lege strings uit de dict
                filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                # en voeg de dictionary toe aan 
                DICT[foi_count] = filtered_dict

                foi_count+=1

            # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
            final_dict = {'resource': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}",
                        'infobox': {'foi_totalDossiers': len(DICT),
                                     'foi_dossiers': DICT}}


            list_of_dicts.append(final_dict)

    number_of_people = 0
    for i in list_of_dicts:
        persons = i['infobox']['foi_totalDossiers']
        number_of_people+=persons
    
    Json = json.dumps(list_of_dicts, indent=4)

    # Write JSON string to a text file
    with open(f"{organisatie_type}_json.txt", "w") as file:
        file.write(Json)
        
    return number_of_people, list_of_dicts


organisatie = 'Gemeenten'
people_num, dict_list = json_create(organisatie)

tree = ET.parse(f"exportOO_{organisatie.lower()}.xml")
namespaces = {
    'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
}
people = tree.findall('.//p:medewerker', namespaces)

print(f'aantal medewerkers in XML: {len(people)}')
print(f'aantal medewerkers extracted in JSON: {people_num}')


identifiers = ['foi_function', 'dc_publisher', 'foaf_initials', 'foaf_firstName', 'foaf_lastName', 'foi_title',
'foaf_mbox', 'foaf_phone', 'foaf_type', 'foi_startDate', 'foi_party']

id_dict = {key: 0 for key in identifiers}

for dl in dict_list:
    dossiers = dl['infobox']['foi_dossiers']
    for k,v in dossiers.items():
        for ID in identifiers:
            if ID in v:
                id_dict[ID] += 1
                
print(id_dict)
    


aantal medewerkers extracted in JSON: 14120
aantal medewerkers in XML: 14120
{'foi_function': 14120, 'dc_publisher': 14120, 'foaf_initials': 14112, 'foaf_firstName': 7, 'foaf_lastName': 14120, 'foi_title': 13998, 'foaf_mbox': 13602, 'foaf_phone': 13758, 'foaf_type': 14120, 'foi_startDate': 0, 'foi_party': 13486}


In [126]:
def json_create_provincies(organisatie_type='Provincies'):
    
    list_of_dicts = []
    
    # afkorting zal later worden gebruikt
    afkortingen = {'Provincies': 'pv'}
    afkorting = afkortingen[organisatie_type]
    
    tree = ET.parse(f"exportOO_{organisatie_type.lower()}.xml")
    
    # Define the namespaces used in the XML document
    namespaces = {
        'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
    }

    # Verzamel de huidige datum
    foi_retrievedDate = datetime.today().strftime('%Y-%m-%d')

    # En het jaar
    dc_date_year = foi_retrievedDate[:4]

    # Verkrijg alle organisatie elementen (gemeenten, waterschappen etc.)
    organisatie_elements = tree.findall('/p:organisaties/p:organisatie', namespaces)

    # loop over deze elementen en verzamel gegevens
    for organisatie in organisatie_elements:

        # In de resourceIdentifierTOOI staat de organisatiecode en veramel deze code
        dc_publisher = xpath_TOOI(organisatie, './/p:identificatiecodes/p:resourceIdentifier[@p:naam="resourceIdentifierTOOI"]', namespaces, afkorting) 
        
        dc_publisher_name = xpath(organisatie, 'p:naam', namespaces)

        website_txt = xpath(organisatie, './/p:contact/p:internetadressen/p:internetadres/p:url''p:types/p:type', namespaces)

        Type = xpath(organisatie, 'p:types/p:type', namespaces)
        
        organisatie_elements2 = organisatie.findall('./p:organisaties/p:organisatie', namespaces)
        
        for organisatie2 in organisatie_elements2:

            # in het functie_element staan alle verschillende functies
            functie_element = organisatie2.findall('p:functies/p:functie', namespaces)

            # loop over deze functies
            for functie in functie_element:
                foi_count = 0
                DICT = dict()

                foaf_function_type = xpath(functie, 'p:naam', namespaces)

                # vindt alle medewerkers die bij deze functie horen
                medewerker_element = functie.findall('p:medewerkers/p:medewerker', namespaces)

                # loop over alle medewerkers om per medewerker gegevens te verzamelen
                for medewerker in medewerker_element:

                    foi_party = xpath(medewerker, 'p:partijLidmaatschap', namespaces)

                    foaf_initials, foaf_firstName, foaf_lastName, name_without_prefix, prefixes = xpath_naam(medewerker, 'p:naam', namespaces)

                    # optioneel startdatum, telefoonnummer en mailadres
                    foi_startDate = xpath(medewerker, 'p:startDatum', namespaces)

                    foaf_phone = xpath(medewerker, 'p:contact/p:telefoonnummers/p:telefoonnummer/p:nummer', namespaces)

                    foaf_mbox = xpath(medewerker, 'p:contact/p:emailadressen/p:emailadres/p:email', namespaces)

            
                    if foi_party != '':
                        bereikbaarheidsgegevens = f"Bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {foi_party} in de {dc_publisher_name}"
                    else:
                        bereikbaarheidsgegevens = f"bereikbaarheidsgegevens van {name_without_prefix}, {foaf_function_type} voor {dc_publisher_name}"

                    # vul de dictionary in
                    Dict = {
                            'dc_identifier': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}.{foi_count + 1}",
                            'dc_title': f"{name_without_prefix} - {dc_publisher_name}",
                            'foi_function': foaf_function_type,
                            'dc_description': bereikbaarheidsgegevens,
                            'dc_source': f"https://organisaties.overheid.nl/archive/exportOO{organisatie_type.lower()}_.xml",
                            'dc_publisher': dc_publisher,
                            'dc_creator': "Ramon Duursma",
                            'foi_retrievedDate': foi_retrievedDate,
                            'dc_date_year': dc_date_year,
                            'dc_publisher_name': dc_publisher_name,
                            'foaf_initials': foaf_initials,
                            'foaf_firstName': foaf_firstName,
                            'foaf_lastName': foaf_lastName,
                            'foi_title': prefixes,
                            'foaf_name': name_without_prefix,
                            'foaf_mbox': foaf_mbox,
                            'foaf_phone': foaf_phone,
                            'foaf_type': Type,
                            'foi_startDate': foi_startDate,
                            'foi_party': foi_party,
                            'foi_files': [] ,
                        }

                    # Verwijder alle lege strings uit de dict
                    filtered_dict = {key: value for key, value in Dict.items() if value != ""}

                    # en voeg de dictionary toe aan 
                    DICT[foi_count] = filtered_dict

                    foi_count+=1

                # uiteindelijk hoeft de dict van alle medewerkers alleen maar toegevoegd te worden aan de volledige dict.
                final_dict = {'resource': f"nl.{dc_publisher}.{foaf_function_type}.{dc_date_year}",
                            'infobox': {'foi_totalDossiers': len(DICT),
                                         'foi_dossiers': DICT}}


                list_of_dicts.append(final_dict)
    
    number_of_people = 0
    for i in list_of_dicts:
        persons = i['infobox']['foi_totalDossiers']
        number_of_people+=persons

    Json = json.dumps(list_of_dicts, indent=4)

    # Write JSON string to a text file
    with open(f"{organisatie_type}_json.txt", "w") as file:
        file.write(Json)
        
    return number_of_people, list_of_dicts

organisatie = 'Provincies'
people_num, dict_list = json_create_provincies(organisatie)

tree = ET.parse(f"exportOO_{organisatie.lower()}.xml")
namespaces = {
    'p': 'https://organisaties.overheid.nl/static/schema/oo/export/2.6.3'
}

people = tree.findall('.//p:medewerker', namespaces)

print(f'aantal medewerkers in XML: {len(people)}')
print(f'aantal medewerkers extracted in JSON: {people_num}')


identifiers = ['foi_function', 'dc_publisher', 'foaf_initials', 'foaf_firstName', 'foaf_lastName', 'foi_title',
'foaf_mbox', 'foaf_phone', 'foaf_type', 'foi_startDate', 'foi_party']

id_dict = {key: 0 for key in identifiers}

for dl in dict_list:
    dossiers = dl['infobox']['foi_dossiers']
    for k,v in dossiers.items():
        for ID in identifiers:
            if ID in v:
                id_dict[ID] += 1
                
print(id_dict)

aantal medewerkers in XML: 340
aantal medewerkers extracted in JSON: 153
{'foi_function': 153, 'dc_publisher': 153, 'foaf_initials': 145, 'foaf_firstName': 21, 'foaf_lastName': 153, 'foi_title': 18, 'foaf_mbox': 127, 'foaf_phone': 29, 'foaf_type': 153, 'foi_startDate': 9, 'foi_party': 121}


C:\Users\ramon\AppData\Local\Temp\ipykernel_5792\2324362.py:23: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './p:organisaties/p:organisatie'
  organisatie_elements = tree.findall('/p:organisaties/p:organisatie', namespaces)


In [163]:
import requests
from bs4 import BeautifulSoup
url = "http://search.yahoo.com/search?p=%s"
query = "remine alberts SP staatslid"
r = requests.get(url % query) 
soup = BeautifulSoup(r.text)
hrefs = [a['href'] for a in soup.find_all('a', href=True)]
hrefs = [href for href in hrefs if 'yahoo' not in hrefs]

new_hrefs = [href for href in hrefs if "yahoo" not in href and '#' not in href]

In [164]:
print(new_hrefs)

['https://amsterdam.sp.nl/onze-mensen/remine-alberts', 'https://cc.bingj.com/cache.aspx?q=remine+alberts+SP+staatslid&d=4666855880851720&mkt=en-US&setlang=en-US&w=4SCAhZ3hgQ5KPYvHpP2IdX8Ip1_onaOe', 'https://amsterdam.sp.nl/nieuws/2022/01/interview-met-lijsttrekker-remine-alberts', 'https://cc.bingj.com/cache.aspx?q=remine+alberts+SP+staatslid&d=4622278410895657&mkt=en-US&setlang=en-US&w=3-NdlQpSIsI1-AzXEmJ8we3fuggiCRQG', 'https://www.amsterdam.nl/bestuur-organisatie/gemeenteraad/fracties/raadsleden-sp/remine-alberts/', 'https://cc.bingj.com/cache.aspx?q=remine+alberts+SP+staatslid&d=4776691082218340&mkt=en-US&setlang=en-US&w=lfvDIwQlKvZ8MFACnEpCIfCeEkj5VXLz', 'https://www.amsterdam.nl/bestuur-organisatie/gemeenteraad/fracties/raadsleden-sp/remine-alberts/', 'https://www.amsterdam.nl/bestuur-organisatie/gemeenteraad/fracties/raadsleden-sp/remine-alberts/', 'https://www.facebook.com/remine.alberts', 'https://twitter.com/RemineAlberts', 'https://www.amsterdam.nl/bestuur-organisatie/gemeen